<a href="https://colab.research.google.com/github/Anavi07/AudioSpotting/blob/main/Bangla_EfficientNet_MS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import cv2
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
# Paths
main_dir = '/content/Bangla-Number-Recognition-CNN'  # Path to FSDD
files_path = '/content/drive/MyDrive/bangla'  # Save processed data

if not os.path.exists(files_path):
    os.makedirs(files_path)

# Clone the dataset if not already cloned
if not os.path.exists(main_dir):
    !git clone https://github.com/ClockWorkKid/Bangla-Number-Recognition-CNN


Cloning into 'Bangla-Number-Recognition-CNN'...
remote: Enumerating objects: 2699, done.
remote: Total 2699 (delta 0), reused 0 (delta 0), pack-reused 2699 (from 1)
Receiving objects: 100% (2699/2699), 85.13 MiB | 38.45 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [4]:
# PROCESSING
# Feature extraction methods

# def CalculateZeroCrossingRate(file_location):
#     y, sr = librosa.load(file_location)
#     zCross = librosa.feature.zero_crossing_rate(y=y)
#     dim = (32, 32)
#     resized = cv2.resize(zCross, dim, interpolation = cv2.INTER_AREA)
#     return resized

def CalculateMelSpectrogram(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.melspectrogram(y=y, sr=sr)
    melSpec_dB = librosa.power_to_db(melSpec)
    dim = (32, 32)
    resized = cv2.resize(melSpec_dB, dim, interpolation = cv2.INTER_AREA)
    return resized


def CalculateMFCC(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.mfcc(y=y, sr=sr)
    dim = (32, 32)
    resized = cv2.resize(melSpec, dim, interpolation = cv2.INTER_AREA)
    return resized



In [5]:
cd /content

/content


In [ ]:
'''for i in range(10):
  j = '/content/dataset'
  print(os.listdir(j))
  for k in os.listdir(j):
    print(k)
    if ".DS_" in k:
      continue

    for t in os.listdir(j+"/"+k):
      #print("path = ", os.listdir(j+"/"+k))
      if ".DS_" in k:
        continue
      print(t.split("WA")[1].split(".")[0])
      #print(t.split("D")[1][0])
      break
    break
  break '''

'for i in range(10):\n  j = \'/content/dataset\'\n  print(os.listdir(j))\n  for k in os.listdir(j):\n    print(k)\n    if ".DS_" in k:\n      continue\n\n    for t in os.listdir(j+"/"+k):\n      #print("path = ", os.listdir(j+"/"+k))\n      if ".DS_" in k:\n        continue\n      print(t.split("WA")[1].split(".")[0])\n      #print(t.split("D")[1][0])\n      break\n    break\n  break '

In [6]:
import os
import numpy as np

# Paths
main_dir = '/content/Bangla-Number-Recognition-CNN'  # Path to the main dataset folder
recordings_dir = os.path.join(main_dir, 'Dataset/Dataset_sorted')  # Path to the Dataset_sorted folder
files_path = '/content/drive/MyDrive/bangla'  # Save processed data

# Ensure output directory exists
if not os.path.exists(files_path):
    os.makedirs(files_path)

# Processing
x = []
y = []

# Loop through each label directory (0, 1, ..., 9, noise)
for label_dir in os.listdir(recordings_dir):
    label_path = os.path.join(recordings_dir, label_dir)

    # Ensure it's a directory
    if not os.path.isdir(label_path):
        continue

    # Process files within the label directory
    for file_name in os.listdir(label_path):
        if not file_name.endswith('.wav'):  # Skip non-wav files
            continue

        file_path = os.path.join(label_path, file_name)

        try:
            # Extract Mel Spectrogram features
            x.append(CalculateMelSpectrogram(file_path))

            # Use directory name as label (e.g., '0', '1', ..., 'noise')
            label = int(label_dir) if label_dir.isdigit() else 10  # Assign 10 for 'noise'
            y.append(label)
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Convert to NumPy arrays
x = np.array(x)
y = np.array(y)

# Verify data
if x.size == 0 or y.size == 0:
    raise ValueError("No data was processed. Check dataset structure and feature extraction.")

# Save processed data
np.save(os.path.join(files_path, "X_melspectrogram.npy"), x)
np.save(os.path.join(files_path, "Y_melspectrogram.npy"), y)

print(f"Processing completed. Processed data saved in {files_path}.")


Processing completed. Processed data saved in /content/drive/MyDrive/bangla.


In [7]:
#load files
files_path = '/content/drive/MyDrive/bangla'  # Save processed data

x_melspectrogram = np.load(files_path+"/X_melspectrogram.npy", allow_pickle=True)
# x_zero_crossing_rate = np.load(files_path+"/X_zero_crossing_rate.npy", allow_pickle=True)
#x_mfcc = np.load(files_path+"/X_mfcc.npy", allow_pickle=True)
y_melspectrogram = np.load(files_path+"/Y_melspectrogram.npy", allow_pickle=True)
# y_zero_crossing_rate = np.load(files_path+"/y_zero_crossing_rate.npy", allow_pickle=True)
#y_mfcc = np.load(files_path+"/Y_mfcc.npy", allow_pickle=True)

In [ ]:
# #Split data into train and test

# from sklearn.model_selection import train_test_split
# x = x_mfcc
# y = y_mfcc

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
# np.save(files_path+"/x_mfcc_train",x_train)
# np.save(files_path+"/y_mfcc_train",y_train)
# np.save(files_path+"/x_mfcc_test",x_test)
# np.save(files_path+"/y_mfcc_test",y_test)

In [8]:
from sklearn.model_selection import train_test_split
x = x_melspectrogram
y = y_melspectrogram

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
np.save(files_path+"/x_melspectrogram_train",x_train)
np.save(files_path+"/y_melspectrogram_train",y_train)
np.save(files_path+"/x_melspectrogram_test",x_test)
np.save(files_path+"/y_melspectrogram_test",y_test)

In [ ]:

# from sklearn.model_selection import train_test_split
# x = x_zero_crossing_rate
# y = y_zero_crossing_rate

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
# np.save(files_path+"/x_zero_crossing_rate_train",x_train)
# np.save(files_path+"/y_zero_crossing_rate_train",y_train)
# np.save(files_path+"/x_zero_crossing_rate_test",x_test)
# np.save(files_path+"/y_zero_crossing_rate_test",y_test)

In [9]:
import tensorflow as tf
import numpy as np

files_path = '/content/drive/MyDrive/bangla'

features = ['melspectrogram', 'mfcc', 'zero_crossing_rate']

x_train = np.load(files_path+"/x_{}_train.npy".format(features[0]), allow_pickle=True)
y_train = np.load(files_path+"/y_{}_train.npy".format(features[0]), allow_pickle=True)

x_test = np.load(files_path+"/x_{}_test.npy".format(features[0]),allow_pickle=True)
y_test = np.load(files_path+"/y_{}_test.npy".format(features[0]),allow_pickle=True)

In [10]:
print(np.unique(y_melspectrogram))

[ 0  1  2  3  4  5  6  7  8  9 10]


In [11]:
x_train =x_train.reshape((x_train.shape[0],32,32,1))
x_test =x_test.reshape((x_test.shape[0],32,32,1))

In [12]:
x_train = x_train.reshape(-1, 32, 32, 1)  # Reshape to match the input shape of the model
x_test = x_test.reshape(-1, 32, 32, 1)    # Reshape for testing
train_y = tf.keras.utils.to_categorical(y_train, num_classes=11)  # One-hot encode
test_y = tf.keras.utils.to_categorical(y_test, num_classes=11)    # One-hot encode


In [13]:
print("Shape of x_train:", x_train.shape)
print("Shape of x_test:", x_test.shape)


Shape of x_train: (1961, 32, 32, 1)
Shape of x_test: (491, 32, 32, 1)


In [14]:
print("Min value in x_train:", x_train.min())
print("Max value in x_train:", x_train.max())


Min value in x_train: -100.00001
Max value in x_train: 36.15473


In [15]:
# Normalize x_train and x_test to the range [0, 1]
x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())


## Train Model

In [17]:
#EfficientNet

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

# Define EfficientNet model for audio data
def GetEfficientNet(input_shape=(32, 32, 1)):
    # Load EfficientNetB0 as base, without top classification layers
    base_model = EfficientNetB0(
        include_top=False,
        input_shape=(32, 32, 3),  # EfficientNet requires 3 channels
        weights=None  # Training from scratch
    )
    base_model.trainable = True  # Allow fine-tuning

    # Add custom layers on top
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),  # Accept audio input shape
        layers.Conv2D(3, (1, 1), activation='relu'),  # Convert single-channel to 3-channel
        base_model,
        layers.GlobalAveragePooling2D(),  # Pooling layer to reduce spatial dimensions
        layers.Dropout(0.5),  # Regularization
        layers.Dense(256, activation='relu'),  # Fully connected layer
        layers.Dropout(0.5),  # Regularization
        layers.Dense(11, activation='softmax')  # Output layer for 10 classes
    ])

    return model

# Preprocess the audio data (assumed to be MFCC or spectrogram)
# x_train and x_test must be 2D arrays representing audio features
x_train = x_train.reshape(-1, 32, 32, 1)  # Reshape to match input shape
x_test = x_test.reshape(-1, 32, 32, 1)    # Same for test data

# Normalize the audio data
x_train = x_train / 255.0
x_test = x_test / 255.0

# One-hot encode the labels for 10-class classification
train_y = tf.keras.utils.to_categorical(y_train, num_classes=11)
test_y = tf.keras.utils.to_categorical(y_test, num_classes=11)

# Compile the EfficientNet model
efficientnet_model = GetEfficientNet(input_shape=(32, 32, 1))
efficientnet_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)  # Lower learning rate
)

# Define callback for saving the best model
callbacks = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/auddsavedfiles/models/efficientnet_model_audio.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Train the model
history = efficientnet_model.fit(
    x_train, train_y,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=0.1,
    callbacks=[callbacks]
)




Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0860 - loss: 3.1159  
Epoch 1: val_accuracy improved from -inf to 0.10152, saving model to /content/drive/MyDrive/auddsavedfiles/models/efficientnet_model_audio.keras
28/28 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.0863 - loss: 3.1119 - val_accuracy: 0.1015 - val_loss: 2.4003
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0971 - loss: 2.7474
Epoch 2: val_accuracy did not improve from 0.10152
28/28 ━━━━━━━━━━━━━━━━━━━━ 49s 26ms/step - accuracy: 0.0974 - loss: 2.7453 - val_accuracy: 0.1015 - val_loss: 2.3989
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1239 - loss: 2.5435
Epoch 3: val_accuracy did not improve from 0.10152
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.1234 - loss: 2.5438 - val_accuracy: 0.1015 - val_loss: 2.3959
Epoch 4/50
26/28 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1080 - loss: 2.5040
Epoch 4: val_accuracy did not improve from 0.10152
28/28 ━━━━━━━

In [18]:
# Evaluate the EfficientNet model
efficientnet_model = tf.keras.models.load_model('/content/drive/MyDrive/auddsavedfiles/models/efficientnet_model_audio.keras')
print("Test Accuracy is ", efficientnet_model.evaluate(x_test, test_y)[1])

16/16 ━━━━━━━━━━━━━━━━━━━━ 12s 295ms/step - accuracy: 0.0874 - loss: 2.4005
Test Accuracy is  0.10794297605752945
